<img src="Bilder/ost_logo.png" width="240"  align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN05/5.4-Padding_und_Strides.ipynb)

In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
%pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt


# Padding and Stride

Im vorherigen Beispiel von hatte unsere Eingabe sowohl eine Höhe als auch eine Breite von 3
und unser Faltungs-Kernel hatte eine Höhe und Breite von 2.
- Daraus ergibt sich eine Ausgabedarstellung mit der Dimension $2\times 2$.

- Unter der Annahme, dass die Eingabeform ist $n_h\times n_w$ und die Form des Faltungskerns ist $k_h\times k_w$, wird die Ausgangsdimension: 

$$(n_h-k_h+1) \times (n_w-k_w+1)$$ 

- Daher wird die Ausgangsform der Faltungsschicht wird also durch die Form der Eingabe und die Form des Faltungskerns bestimmt.




### Faltungskerne reduzieren i.A. die Ausgabedimension

- Kernel haben im Allgemeinen eine Breite und Höhe von mehr als $1$, und nach der Anwendung vieler aufeinander folgender Faltungen neigen CNN dazu, mit Ausgaben zu enden, die wesentlich kleiner sind als unsere Eingabe. 
- Wenn wir mit einem Bild mit $240 \times 240$ Pixeln beginnen, reduzieren $10$ Schichten von $5 \times 5$ Faltungen das Bild auf $200 \times 200$ Pixel, wodurch $30 \%$ des Bildes abgeschnitten werden und damit werden alle interessanten Informationen an den Rändern des ursprünglichen Bildes abgeschnitten.

- **Padding** ist das gängigste Werkzeug zur Behandlung dieses Problems.
- In anderen Fällen möchten wir vielleicht die Dimensionalität drastisch reduzieren, z. B. wenn wir die ursprüngliche Eingabeauflösung als unhandlich empfinden. Die **Strided Convolutions** sind eine beliebte Technik, die in solchen Fällen helfen kann.

## Padding (Auffüllen)

Wie oben beschrieben, besteht ein heikles Problem bei der Anwendung von Faltungsschichten
ist, dass wir dazu neigen, Pixel an den Rändern unseres Bildes zu verlieren.

- Da wir normalerweise kleine Kernel verwenden, gehen für jede einzelne Faltungschicht vielleicht nur ein paar Pixel verloren, aber das kann sich summieren, wenn wir viele aufeinanderfolgende Faltungsschichten hintereinander stapeln.
- Eine unkomplizierte Lösung für dieses Problem ist das Hinzufügen *zusätzlicher Füllpixel am Rande* des Eingabebildes, und so die effektive Grösse des Bildes zu erhöhen.
- Normalerweise setzen wir die Werte der zusätzlichen Pixel auf `Null`.




In folgendem Beispiel (siehe Abbildung) füllen wir eine $3 \times 3$ Eingabe auf, und erhöhen seine Größe auf $5 \times 5$. 
- Die entsprechende Ausgabe erhöht sich dann auf eine $4 \times 4$-Matrix.
- Die schattierten Bereiche sind das erste Ausgabeelement sowie die Eingabe- und Kernel-Tensorelemente, die für die Ausgabeberechnung verwendet werden: $0 \cdot 0+0 \cdot 1+0 \cdot 2+0 \cdot 3=0$.

<img src="Bilder/conv-pad.svg" width="540" height="440" align="center"/>

Im Allgemeinen, wenn wir insgesamt $p_h$ Zeilen der Auffüllung hinzufügen (ungefähr die Hälfte oben und die Hälfte unten)
und insgesamt $p_w$ Spalten von padding (ungefähr die Hälfte auf der linken und die Hälfte auf der rechten Seite),
wird die Ausgabeform sein

$$\boxed{(n_h-k_h+p_h+1)\times(n_w-k_w+p_w+1)}.$$

Das bedeutet, dass die Höhe und Breite der Ausgabe
um $p_h$ bzw. $p_w$ zunehmen werden.

- In vielen Fällen werden wir $p_h=k_h-1$ und $p_w=k_w-1$ setzen wollen, damit die Eingabe und die Ausgabe die gleiche Höhe und Breite haben.
- Dies erleichtert die Vorhersage der Ausgangsform jeder Schicht bei der Konstruktion des Netzes.
- Nehmen wir an, dass $k_h$ hier ungerade ist, werden wir $p_h/2$ Zeilen auf beiden Seiten der Höhe auffüllen.
- Wenn $k_h$ gerade ist, besteht eine Möglichkeit darin $\lceil p_h/2\rceil$ Zeilen auf der Oberseite der Eingabe aufzufüllen und $\lfloor p_h/2\rfloor$ Zeilen an der Unterseite.
- Wir werden beide Seiten der Breite auf die gleiche Weise auffüllen.



### CNNs verwenden meist Faltungs-Kerne mit *ungeraden* Höhen und Breiten

-CNNs verwenden in der Regel Faltungs-Kerne mit **ungeraden Höhen- und Breitenwerten**, wie 1, 3, 5 oder 7.
- Die Wahl von ungeraden Kernelgrössen hat den Vorteil dass wir die räumliche Dimensionalität erhalten können und gleichzeitig die gleiche Anzahl von Zeilen oben und unten aufzufüllen, und die gleiche Anzahl von Spalten links und rechts.
- Ausserdem hat diese Praxis der Verwendung ungerader Kernel und Auffüllung zur genauen Erhaltung der Dimensionalität einen verwaltungstechnischen Vorteil.

Wenn die Grösse des Kerns ungerade ist und die Anzahl der Auffüllungszeilen und -spalten auf allen Seiten gleich ist, wird für jeden zweidimensionalen Tensor `X`  eine Ausgabe mit der gleichen Höhe und Breite wie die Eingabe erzeugt.
Somit wissen wir, dass die Ausgabe `Y[i, j]` berechnet wird durch Kreuzkorrelation von Eingabe und Faltungskern
mit dem Fenster, das auf `X[i, j]` zentriert ist.

Im folgenden Beispiel erstellen wir eine zweidimensionale Faltungsschicht mit einer Höhe und Breite von 3
und **an allen Seiten 1 Pixel Auffüllung**. Bei einer Eingabe mit einer Höhe und Breite von 8, stellen wir fest, dass die Höhe und Breite der Ausgabe ebenfalls 8 beträgt.

In [ ]:
import tensorflow as tf


# We define a convenience function to calculate the convolutional layer. This
# function initializes the convolutional layer weights and performs
# corresponding dimensionality elevations and reductions on the input and
# output
def comp_conv2d(conv2d, X):
    # Here (1, 1) indicates that the batch size and the number of channels
    # are both 1
    X = tf.reshape(X, (1,) + X.shape + (1,))
    Y = conv2d(X)
    # Exclude the first two dimensions that do not interest us: examples and
    # channels
    return tf.reshape(Y, Y.shape[1:3])


# Note that here 1 row or column is padded on either side, so a total of 2
# rows or columns are added
conv2d = tf.keras.layers.Conv2D(1, kernel_size=3, padding="same")
X = tf.random.uniform(shape=(8, 8))
comp_conv2d(conv2d, X).shape

Wenn die Höhe und Breite des Faltungskerns unterschiedlich sind,
können wir dafür sorgen, dass die Ausgabe und die Eingabe die gleiche Höhe und Breite haben
indem man **unterschiedliche Auffüllungszahlen (paddings) für Höhe und Breite einstellt werden.**


In [ ]:
# Here, we use a convolution kernel with a height of 5 and a width of 3. The
# padding numbers on either side of the height and width are 2 and 1,
# respectively
conv2d = tf.keras.layers.Conv2D(1, kernel_size=(5, 3), padding="same")
comp_conv2d(conv2d, X).shape

## Stride (Schrittweite)

Bei der Berechnung der Kreuzkorrelation beginnen wir mit dem Faltungsfenster in der oberen linken Ecke des Eingabetensors,
und schieben es dann über alle Stellen nach unten und nach rechts.
- In den vorherigen Beispielen haben wir standardmäßig ein Element nach dem anderen verschoben.
- Manchmal jedoch, entweder aus Gründen der *Berechnungseffizienz* oder weil wir ein *Downsampling* durchführen wollen,
verschieben wir unser Fenster mehr als ein Element auf einmal, wobei die **Zwischenpositionen übersprungen** werden.

Die Anzahl der Zeilen und Spalten, die pro Folie durchlaufen werden, bezeichnet man als *Stride* (Schrittweite).
- Bisher haben wir sowohl für die Höhe als auch für die Breite eine Schrittweite von 1 verwendet.
- In manchen Fällen kann es sinnvoll sein, eine grössere Schrittweite zu verwenden.




Die folgende Abbildung zeigt eine zweidimensionale Kreuzkorrelationsoperation mit einer Schrittweite von 3 in der Vertikalen und 2 in der Horizontalen. 
- Die schattierten Bereiche sind die Ausgabeelemente sowie die Eingabe- und Kernel-Tensorelemente, die für die Ausgabeberechnung verwendet werden: $0\cdot 0+0\cdot 1+1\cdot 2+2\cdot 3=8$, $0\cdot0+6\cdot1+0\cdot2+0\cdot3=6$.
- Wir sehen, dass, wenn das zweite Element der ersten Spalte ausgegeben wird, das Faltungsfenster drei Zeilen nach unten gleitet.
- Das Faltungsfenster verschiebt sich um zwei Spalten nach rechts wenn das zweite Element der ersten Zeile ausgegeben wird.
- Wenn das Faltungsfenster bei der Eingabe weiterhin zwei Spalten nach rechts rutscht, gibt es keine Ausgabe, weil das Eingabeelement das Fenster nicht ausfüllen kann nicht ausfüllen kann (es sei denn, wir fügen eine weitere Spalte als Füllung hinzu).

<img src="Bilder/conv-stride.svg" width="440" height="340" align="center"/>

                        Kreuzkorrelation mit Schrittweiten von 3 und 2 für Höhe bzw. Breite

Für eine Schrittweite für die Höhe $s_h$ für die Breite $s_w$  ist die Ausgabedimension:

$$\boxed{\lfloor(n_h-k_h+p_h+s_h)/s_h\rfloor \times \lfloor(n_w-k_w+p_w+s_w)/s_w\rfloor}.$$

Wenn wir $p_h=k_h-1$ und $p_w=k_w-1$ setzen, dann wird die Ausgangsdimension vereinfacht zu:

$$\lfloor(n_h+s_h-1)/s_h\rfloor \times \lfloor(n_w+s_w-1)/s_w\rfloor$$.


Wenn die Höhe und Breite der Eingabe durch die Schrittweite der Höhe und Breite teilbar sind, dann wird die Ausgabeform vereinfacht zu:

$$(n_h/s_h) \times (n_w/s_w)$$.

Im Folgenden wird **die Schrittweite für Höhe und Breite auf 2** gesetzt, und somit wird die Eingabehöhe und -breite jeweils halbiert.


In [ ]:
conv2d = tf.keras.layers.Conv2D(1, kernel_size=3, padding="same", strides=2)
comp_conv2d(conv2d, X).shape

Als Nächstes werden wir uns **ein etwas komplizierteres Beispiel** ansehen.

In [ ]:
conv2d = tf.keras.layers.Conv2D(1, kernel_size=(3, 5), padding="valid", strides=(3, 4))
comp_conv2d(conv2d, X).shape


Der Einfachheit halber nennen wir die Auffüllungszahl auf beiden Seiten der Eingabehöhe und -breite $p_h$ bzw. $p_w$ ist, nennen wir die Auffüllung $(p_h, p_w)$.
- Genauer gesagt, wenn $p_h = p_w = p$, ist die Auffüllung $p$.
- Wenn die Schrittweiten in der Höhe und Breite $s_h$ bzw. $s_w$ sind, nennen wir die Schrittweite $(s_h, s_w)$.
- Genauer gesagt, wenn $s_h = s_w = s$ ist, ist die Schrittweite $s$.
- **Standardmässig ist das Padding 0 und der Stride 1.**
- In der Praxis verwenden wir selten inhomogene Schrittweiten oder Auffüllungen, d.h. wir haben normalerweise $p_h = p_w$ und $s_h = s_w$.**

## Zusammenfassung

* Durch Auffüllen kann die Höhe und Breite der Ausgabe vergrößert werden. Dies wird oft verwendet, um der Ausgabe die gleiche Höhe und Breite wie der Eingabe zu geben.
* Stride kann die Auflösung der Ausgabe reduzieren, z.B. die Höhe und Breite der Ausgabe auf nur $1/n$ der Höhe und Breite der Eingabe reduzieren ($n$ ist eine ganze Zahl größer als $1$).
* Padding und Stride können verwendet werden, um die Dimensionalität der Daten effektiv anzupassen.




## Übungen (optional)

1. Berechnen Sie für das letzte Beispiel in diesem Abschnitt die Form der Ausgabe mit Hilfe der Mathematik, um zu sehen, ob sie mit dem experimentellen Ergebnis übereinstimmt.
2. Probieren Sie andere Kombinationen von Padding und Stride für die Experimente in diesem Abschnitt aus.
3. Was bedeutet bei Audiosignalen eine Schrittweite von 2?
4. Was sind die rechnerischen Vorteile eines Stride grösser als 1?